### **Word2Vec Encoding Method**

In [1]:
import pandas as pd

# Load the dataset
# Replace with the actual path to your CSV file
df = pd.read_csv('/content/cleaned_dataset_updated.csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())




Original DataFrame:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      1   
1           1      3            0                   3        0      0   
2           2      3            0                   3        0      0   
3           3      3            0                   2        1      0   
4           4      6            0                   6        0      0   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  ['retweet', 'woman', 'complain', 'cleaning', '...  
1  ['retweet', 'boy', 'dat', 'coldtyga', 'bad', '...  
2  ['retweet', 'dawg', 'retweet', 'ever

In [2]:
import ast

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Display the first few rows to verify
print("\nDataFrame after converting tweet_tokens to lists:")
print(df.head())



DataFrame after converting tweet_tokens to lists:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      1   
1           1      3            0                   3        0      0   
2           2      3            0                   3        0      0   
3           3      3            0                   2        1      0   
4           4      6            0                   6        0      0   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  [retweet, woman, complain, cleaning, house, an...  
1  [retweet, boy, dat, coldtyga, bad, cuffin, dat...  
2  [retw

In [3]:
from gensim.models import Word2Vec

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tweet_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Get the vector size
vector_size = word2vec_model.wv.vector_size



In [4]:
import numpy as np

def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # If the document is empty, return a zero vector
    if not doc:
        return np.zeros(vector_size)
    # Calculate the mean vector
    return np.mean(word2vec_model.wv[doc], axis=0)

# Apply the function to transform tweet_tokens
X_vectors = df['tweet_tokens'].apply(document_vector).tolist()
X_vectors = np.array(X_vectors)

# Display the first few rows to verify
print("\nVectorized DataFrame:")
print(X_vectors)
print(X_vectors.shape)




Vectorized DataFrame:
[[-0.14082471  0.44016486  0.06225185 ... -0.62547803  0.13013999
  -0.54296857]
 [-0.1067239   0.36742252 -0.00740037 ... -0.50695616  0.13325049
  -0.49256691]
 [-0.13244572  0.42608422 -0.01693738 ... -0.57374227  0.1624656
  -0.55834991]
 ...
 [ 0.00891974  0.31136045  0.01564116 ... -0.54785264  0.30374619
  -0.839293  ]
 [-0.0532313   0.26881793 -0.00480163 ... -0.37872431  0.10320554
  -0.35580322]
 [-0.06812277  0.19264875  0.0409341  ... -0.27052113  0.04469281
  -0.20821074]]
(24783, 100)


In [5]:
# Combine Word2Vec features with other features
other_features = df[['tweet_tokens']]
X_combined = np.hstack((other_features, X_vectors))

# Define the target variable
y = df['class']

# Display the first few rows to verify
print("\nCombined DataFrame:")
print(X_combined)
print(X_combined.shape)


Combined DataFrame:
[[list(['retweet', 'woman', 'complain', 'cleaning', 'house', 'andamp', 'man', 'always', 'take', 'trash'])
  -0.14082470536231995 0.44016486406326294 ... -0.6254780292510986
  0.13013999164104462 -0.5429685711860657]
 [list(['retweet', 'boy', 'dat', 'coldtyga', 'bad', 'cuffin', 'dat', 'hoe', 'st', 'place'])
  -0.10672390460968018 0.36742252111434937 ... -0.506956160068512
  0.13325048983097076 -0.49256691336631775]
 [list(['retweet', 'dawg', 'retweet', 'ever', 'fuck', 'bitch', 'start', 'cry', 'confused', 'shit'])
  -0.13244572281837463 0.4260842204093933 ... -0.5737422704696655
  0.16246560215950012 -0.5583499073982239]
 ...
 [list(['young', 'buck', 'wan', 'na', 'eat', 'dat', 'nigguh', 'like', 'fuckin', 'dis'])
  0.008919736370444298 0.3113604485988617 ... -0.547852635383606
  0.30374619364738464 -0.8392930030822754]
 [list(['youu', 'got', 'wild', 'bitch', 'tellin', 'lie'])
  -0.05323129519820213 0.2688179314136505 ... -0.3787243068218231
  0.10320553928613663 -0.35